In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import glob
from radprompter import RadPrompter, Prompt, vLLMClient

In [3]:
sample_report = """
Clinical Information:
67-year-old male with shortness of breath and pleuritic chest pain. Rule out pulmonary embolism.

Technique:
CT pulmonary angiography with IV contrast was performed.

Findings:
There are filling defects within the right and left main pulmonary arteries extending into the lobar branches, compatible with bilateral pulmonary emboli. No evidence of right heart strain.

Ground glass opacities are present within the peripheral lungs bilaterally, suggesting a component of hemorrhage.

The main pulmonary arteries are dilated, with a diameter of 3.4 cm on the right and 3.1 cm on the left (upper limits of normal).

No pleural effusion is identified.

Impression:
1. Bilateral pulmonary emboli involving the right and left main pulmonary arteries and lobar branches.
2. Findings suggestive of pulmonary hemorrhage.
3. Pulmonary artery dilation suggesting chronic pulmonary hypertension.

Recommendation:
Clinical correlation is recommended. Anticoagulation therapy should be initiated if not already started. Consider echocardiography to evaluate for right heart strain.
"""

In [4]:
prompt = Prompt('sample_prompt.toml')

In [5]:
prompt

In [7]:
client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-70B-Instruct",
    base_url = "http://localhost:8000/v1",
    temperature = 0.0,
    seed=42
)

In [52]:
prompt.schemas

[{'variable_name': 'Pulmonary Embolism',
  'type': 'select',
  'options': ['Present', 'Absent'],
  'hint': '\'Pulmonary Embolism\'\nHere are your options and you can explicitly use one of these:\n  - `Present`\n  - `Absent`\n\nHint: "Indicate `Present` if the report explicitly mentions the patient has pulmonary embolism in their CT scan. \nIndicate `Absent` if pulmonary embolism is not seen or if a previously observed pulmonary embolism is mentioned as resolved.'},
 {'variable_name': 'Laterality',
  'type': 'select',
  'options': ['Left', 'Right', 'Bilateral', 'Not Mentioned'],
  'hint': "'Laterality'\nHere are your options and you can explicitly use one of these:\n  - `Left`\n  - `Right`\n  - `Bilateral`\n  - `Not Mentioned`\n\nHint: Indicate `Left` if the report explicitly mentions the patient has a left-sided pulmonary embolism in their CT scan. \nIndicate `Right` if the report explicitly mentions the patient has a right-sided pulmonary embolism in their CT scan. \nIndicate `Bilater

In [53]:
engine = RadPrompter(
    client=client, 
    prompt=prompt,
    concurrency=12,
    hide_blocks=False,
)

In [54]:
sample = {
    "report": sample_report,
}
engine(sample)

Processing items:   0%|          | 0/1 [00:00<?, ?it/s]

Processing items: 100%|██████████| 1/1 [00:20<00:00, 20.62s/it]


[[{'Pulmonary Embolism': ['<initial_answer>\nPresent\n</initial_answer>\n<explanation>\n1. The report explicitly states "filling defects within the right and left main pulmonary arteries extending into the lobar branches, compatible with bilateral pulmonary emboli" in the Findings section.\n2. This indicates that pulmonary embolism is present in the patient\'s CT scan.\n</explanation>\n<final_answer>\nPresent\n</final_answer>\n',
    '<json>\n{\n  "Pulmonary Embolism" : "Present"\n}\n']},
  {'Laterality': ['<answer>\n<initial_answer>\nBilateral\n</initial_answer>\n<explanation>\n1. The report states "filling defects within the right and left main pulmonary arteries extending into the lobar branches, compatible with bilateral pulmonary emboli" in the Findings section.\n2. This indicates that the patient has pulmonary emboli in both right and left sides.\n</explanation>\n<final_answer>\nBilateral\n</final_answer>\n',
    '<json>\n{\n  "Laterality" : "Bilateral"\n}\n']},
  {'Acuity': ['<a

In [56]:
engine.save_log()